# Blablattractivity - Local and tests -

## Data collection

We start with basic imports to use blablacar web api.

In [1]:
#%pip install pandas

In [2]:
#%pip install fsspec

In [3]:
#%pip install google-cloud-storage

In [4]:
#%pip install matplotlib

In [5]:
#%pip install seaborn

In [2]:
import json
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import matplotlib.dates as mdates
import seaborn as sns
from google.cloud import storage
import datetime 
from math import ceil, floor

In [17]:
origin_destination_df = pd.read_csv("./origin_destination.csv", sep=";") 
route_dict = origin_destination_df.to_dict('index') #'index'
route_dict 
for index in route_dict:
    print('Origin',route_dict[index]['Origin'])
    print('Destination',route_dict[index]['Destination'])
    print('From_coordinate',route_dict[index]['From_coordinate'])
    print('To_coordinate',route_dict[index]['To_coordinate'])
    print("----------------------------------------------------")

Origin Paris
Destination Brest
From_coordinate 48.856614%7C2.352221
To_coordinate 48.390394%7C-4.486076
----------------------------------------------------
Origin Paris
Destination Aix-en-Provence
From_coordinate 48.856614%7C2.352221
To_coordinate 43.529742%2C5.447426
----------------------------------------------------
Origin Paris
Destination Le Mans
From_coordinate 48.856614%7C2.352221
To_coordinate 48.00611%2C0.199556
----------------------------------------------------
Origin Paris
Destination Clermont-Ferrand
From_coordinate 48.856614%7C2.352221
To_coordinate 45.777221%2C3.087025
----------------------------------------------------
Origin Paris
Destination Villeurbanne
From_coordinate 48.856614%7C2.352221
To_coordinate 45.771944%2C4.89017
----------------------------------------------------
Origin Paris
Destination Nîmes
From_coordinate 48.856614%7C2.352221
To_coordinate 43.836699%2C4.360054
----------------------------------------------------
Origin Paris
Destination Angers
Fro

Destination Le Mans
From_coordinate 45.771944%2C4.89017
To_coordinate 48.00611%2C0.199556
----------------------------------------------------
Origin Villeurbanne
Destination Clermont-Ferrand
From_coordinate 45.771944%2C4.89017
To_coordinate 45.777221%2C3.087025
----------------------------------------------------
Origin Villeurbanne
Destination Nîmes
From_coordinate 45.771944%2C4.89017
To_coordinate 43.836699%2C4.360054
----------------------------------------------------
Origin Villeurbanne
Destination Angers
From_coordinate 45.771944%2C4.89017
To_coordinate 47.471161%2C-0.551826
----------------------------------------------------
Origin Villeurbanne
Destination Dijon
From_coordinate 45.771944%2C4.89017
To_coordinate 47.32204%2C5.0414
----------------------------------------------------
Origin Villeurbanne
Destination Grenoble
From_coordinate 45.771944%2C4.89017
To_coordinate 45.18852%2C5.72452
----------------------------------------------------
Origin Villeurbanne
Destination Le H

To_coordinate 49.258329%2C4.031696
----------------------------------------------------
Origin Angers
Destination Rennes
From_coordinate 47.471161%2C-0.551826
To_coordinate 48.117266%2C-1.677793
----------------------------------------------------
Origin Angers
Destination Lille
From_coordinate 47.471161%2C-0.551826
To_coordinate 50.62925%2C3.057256
----------------------------------------------------
Origin Angers
Destination Bordeaux
From_coordinate 47.471161%2C-0.551826
To_coordinate 44.837789%2C-0.57918
----------------------------------------------------
Origin Angers
Destination Strasbourg
From_coordinate 47.471161%2C-0.551826
To_coordinate 48.573405%2C7.752111
----------------------------------------------------
Origin Angers
Destination Montpellier
From_coordinate 47.471161%2C-0.551826
To_coordinate 43.61076%2C3.87671
----------------------------------------------------
Origin Angers
Destination Nantes
From_coordinate 47.471161%2C-0.551826
To_coordinate 47.218371%2C-1.553621
--

Destination Marseille
From_coordinate 45.18852%2C5.72452
To_coordinate 43.296482%2C5.36978
----------------------------------------------------
Origin Grenoble
Destination Paris
From_coordinate 45.18852%2C5.72452
To_coordinate 48.856614%7C2.352221
----------------------------------------------------
Origin Le Havre
Destination Brest
From_coordinate 49.49437%2C0.107929
To_coordinate 48.390394%7C-4.486076
----------------------------------------------------
Origin Le Havre
Destination Aix-en-Provence
From_coordinate 49.49437%2C0.107929
To_coordinate 43.529742%2C5.447426
----------------------------------------------------
Origin Le Havre
Destination Le Mans
From_coordinate 49.49437%2C0.107929
To_coordinate 48.00611%2C0.199556
----------------------------------------------------
Origin Le Havre
Destination Clermont-Ferrand
From_coordinate 49.49437%2C0.107929
To_coordinate 45.777221%2C3.087025
----------------------------------------------------
Origin Le Havre
Destination Villeurbanne
Fro

Destination Toulon
From_coordinate 45.439695%2C4.387177
To_coordinate 43.124228%2C5.928
----------------------------------------------------
Origin Saint-Etienne
Destination Reims
From_coordinate 45.439695%2C4.387177
To_coordinate 49.258329%2C4.031696
----------------------------------------------------
Origin Saint-Etienne
Destination Rennes
From_coordinate 45.439695%2C4.387177
To_coordinate 48.117266%2C-1.677793
----------------------------------------------------
Origin Saint-Etienne
Destination Lille
From_coordinate 45.439695%2C4.387177
To_coordinate 50.62925%2C3.057256
----------------------------------------------------
Origin Saint-Etienne
Destination Bordeaux
From_coordinate 45.439695%2C4.387177
To_coordinate 44.837789%2C-0.57918
----------------------------------------------------
Origin Saint-Etienne
Destination Strasbourg
From_coordinate 45.439695%2C4.387177
To_coordinate 48.573405%2C7.752111
----------------------------------------------------
Origin Saint-Etienne
Destinati

Destination Nantes
From_coordinate 48.117266%2C-1.677793
To_coordinate 47.218371%2C-1.553621
----------------------------------------------------
Origin Rennes
Destination Nice
From_coordinate 48.117266%2C-1.677793
To_coordinate 43.710172%2C7.261953
----------------------------------------------------
Origin Rennes
Destination Toulouse
From_coordinate 48.117266%2C-1.677793
To_coordinate 43.604652%2C1.444209
----------------------------------------------------
Origin Rennes
Destination Lyon
From_coordinate 48.117266%2C-1.677793
To_coordinate 45.76404%2C4.83565
----------------------------------------------------
Origin Rennes
Destination Marseille
From_coordinate 48.117266%2C-1.677793
To_coordinate 43.296482%2C5.36978
----------------------------------------------------
Origin Rennes
Destination Paris
From_coordinate 48.117266%2C-1.677793
To_coordinate 48.856614%7C2.352221
----------------------------------------------------
Origin Lille
Destination Brest
From_coordinate 50.62925%2C3.05

To_coordinate 45.777221%2C3.087025
----------------------------------------------------
Origin Strasbourg
Destination Villeurbanne
From_coordinate 48.573405%2C7.752111
To_coordinate 45.771944%2C4.89017
----------------------------------------------------
Origin Strasbourg
Destination Nîmes
From_coordinate 48.573405%2C7.752111
To_coordinate 43.836699%2C4.360054
----------------------------------------------------
Origin Strasbourg
Destination Angers
From_coordinate 48.573405%2C7.752111
To_coordinate 47.471161%2C-0.551826
----------------------------------------------------
Origin Strasbourg
Destination Dijon
From_coordinate 48.573405%2C7.752111
To_coordinate 47.32204%2C5.0414
----------------------------------------------------
Origin Strasbourg
Destination Grenoble
From_coordinate 48.573405%2C7.752111
To_coordinate 45.18852%2C5.72452
----------------------------------------------------
Origin Strasbourg
Destination Le Havre
From_coordinate 48.573405%2C7.752111
To_coordinate 49.49437%2C

Destination Rennes
From_coordinate 47.218371%2C-1.553621
To_coordinate 48.117266%2C-1.677793
----------------------------------------------------
Origin Nantes
Destination Lille
From_coordinate 47.218371%2C-1.553621
To_coordinate 50.62925%2C3.057256
----------------------------------------------------
Origin Nantes
Destination Bordeaux
From_coordinate 47.218371%2C-1.553621
To_coordinate 44.837789%2C-0.57918
----------------------------------------------------
Origin Nantes
Destination Strasbourg
From_coordinate 47.218371%2C-1.553621
To_coordinate 48.573405%2C7.752111
----------------------------------------------------
Origin Nantes
Destination Montpellier
From_coordinate 47.218371%2C-1.553621
To_coordinate 43.61076%2C3.87671
----------------------------------------------------
Origin Nantes
Destination Nice
From_coordinate 47.218371%2C-1.553621
To_coordinate 43.710172%2C7.261953
----------------------------------------------------
Origin Nantes
Destination Toulouse
From_coordinate 47.

To_coordinate 48.856614%7C2.352221
----------------------------------------------------
Origin Lyon
Destination Brest
From_coordinate 45.76404%2C4.83565
To_coordinate 48.390394%7C-4.486076
----------------------------------------------------
Origin Lyon
Destination Aix-en-Provence
From_coordinate 45.76404%2C4.83565
To_coordinate 43.529742%2C5.447426
----------------------------------------------------
Origin Lyon
Destination Le Mans
From_coordinate 45.76404%2C4.83565
To_coordinate 48.00611%2C0.199556
----------------------------------------------------
Origin Lyon
Destination Clermont-Ferrand
From_coordinate 45.76404%2C4.83565
To_coordinate 45.777221%2C3.087025
----------------------------------------------------
Origin Lyon
Destination Villeurbanne
From_coordinate 45.76404%2C4.83565
To_coordinate 45.771944%2C4.89017
----------------------------------------------------
Origin Lyon
Destination Nîmes
From_coordinate 45.76404%2C4.83565
To_coordinate 43.836699%2C4.360054
------------------

Dès qu'on reçoit l'API on part là-dessus : https://dev.blablacar.com/hc/en-us/articles/360012064820--Search-V3-API-Documentation

In [2]:
api_token = open("./api.txt","r").read()
api_url = "https://public-api.blablacar.com"

We have the basic parameters of our API call :

In [3]:
from_country = "FR" # string : ISO_3166_1
to_country = "FR" # string : ISO_3166_1
locale = "fr-FR" # string : locale in which the result will be displayed https://dev.blablacar.com/hc/en-us/articles/360008930020
currency = "EUR" # string : ISO_4217 currency
number_of_trips_we_want_returned = "100" # int : from 1 to 100, default is 10 
requested_seats = "1" # int : minimum number of seats requested, default value is 1
radius_in_meters = "6000" # int : radius of the circle around the coordinates in which trips will be searched from (in meters) - the default is 5000, so I chose 6000 because I noticed that 5000 is okay for cities but I want a margin of error for people meeting a bit away from cities

Then the date of the trip will be different everytime we run the code (it has to be next friday everytime), so we compute it here :

In [4]:
today = datetime.date.today()
friday = today + datetime.timedelta( (4-today.weekday()) % 7 )
next_friday_datetime = str(friday)+"T00:00:00"
start_date_local = next_friday_datetime

Create the dictionnary that is going to hold our information :

In [ ]:
new_dict = {'trip_id':[],'price_of_the_trip':[],'date':[],'origin':[],'destination':[]}

Collect data : 

In [10]:
for index in route_dict:
    current_origin = route_dict[index]['Origin']
    current_destination = route_dict[index]['Destination']
    from_coordinate = route_dict[index]['From_coordinate'] # string : latitude, longitude - current_origin
    to_coordinate = route_dict[index]['To_coordinate'] # string : latitude, longitude - current_destination
    blabla_query = api_url+"/api/v3/trips?key="+api_token+"&from_coordinate="+from_coordinate+"&from_country="
                            +from_country+"&to_coordinate="+to_coordinate+"&to_country="+to_country+"&locale="+locale+"&currency="
                            +currency+"&count="+number_of_trips_we_want_returned+"&start_date_local="+start_date_local
                            +"&radius_in_meters="+radius_in_meters+"&sort=price:asc"
    response = requests.get(blabla_query)
    # print(response.status_code) # SHOULD BE 200
    json_trips = response.json() # it's a dictionnary
    for dict_trip in json_trips["trips"]:
        trip_id =  dict_trip["link"].split("id=")[1].split("-")[0]
        price_of_the_trip = dict_trip["price"]["amount"]
        new_dict['trip_id'].append(trip_id)
        new_dict['price_of_the_trip'].append(price_of_the_trip)
        new_dict['date'].append(start_date_local[:10])
        new_dict['origin'].append(current_origin)
        new_dict['destination'].append(current_destination)

Feed it to the dataframe :

In [14]:
new_df = pd.DataFrame.from_dict(new_dict)
new_df.price_of_the_trip = new_df.price_of_the_trip.astype('float')    

Save the df in folder here :

In [15]:
old_df = pd.read_csv("./blablacar_rennes_brest.csv",index_col = "Unnamed: 0") 
old_df.head()
df_trips = old_df.append(new_df, ignore_index=True, sort=False)
df_trips.head()
df_trips.to_csv("./blablacar_rennes_brest.csv")